
# Oil & Gas — EDA Notebook (H6.S1)

This notebook performs **Exploratory Data Analysis** over the marts created in dbt:
- `MARTS.INT_ANNUAL_PRODUCTION_DEDUP`
- `MARTS.DIM_WELL`
- `MARTS.DIM_YEAR`

It can read **directly from Snowflake** (recommended) or **from a CSV fallback** exported previously.
Artifacts (plots & CSV summaries) are written under `docs/eda/`.


## Prerequisites
- Python libs: `pandas`, `matplotlib`, `python-dotenv`, `snowflake-connector-python`
- A `.env` file at root with:
```
SNOWFLAKE_ACCOUNT=...
SNOWFLAKE_USER=...
SNOWFLAKE_PASSWORD=...
SNOWFLAKE_ROLE=ROLE_OILGAS_SVC
SNOWFLAKE_WAREHOUSE=COMPUTE_WH
SNOWFLAKE_DATABASE=OILGAS_DB
```


In [ ]:

import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# --- Settings
USE_SNOWFLAKE = True        # Set to False to use CSV fallback without connecting
CSV_FALLBACK = "docs/eda/int_annual_production_for_eda.csv"
EXPORT_DIR   = "../docs/eda"

os.makedirs(EXPORT_DIR, exist_ok=True)
print("Artifacts will be saved to:", EXPORT_DIR)


Artifacts will be saved to: ../docs/eda


In [7]:

from dotenv import load_dotenv
load_dotenv()

def _snowflake_available():
    try:
        import snowflake.connector  # noqa
        return True
    except Exception as _e:
        return False

def get_snowflake_conn():
    import snowflake.connector
    return snowflake.connector.connect(
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
        user=os.getenv("SNOWFLAKE_USER"),
        password=os.getenv("SNOWFLAKE_PASSWORD"),
        role=os.getenv("SNOWFLAKE_ROLE", "ROLE_OILGAS_SVC"),
        warehouse=os.getenv("SNOWFLAKE_WAREHOUSE", "COMPUTE_WH"),
        database=os.getenv("SNOWFLAKE_DATABASE", "OILGAS_DB"),
        schema="MARTS",
    )

def load_data():
    """Load INT table for EDA. Falls back to CSV if Snowflake not available/desired."""
    if USE_SNOWFLAKE and _snowflake_available():
        try:
            with get_snowflake_conn() as conn:
                q = """
                    select *
                    from INT_ANNUAL_PRODUCTION_DEDUP
                """
                df = pd.read_sql(q, conn)
                # Normalize column names to lower case
                df.columns = [c.lower() for c in df.columns]
                # Persist a CSV snapshot for offline runs
                out_csv = Path(EXPORT_DIR) / "int_annual_production_for_eda.csv"
                df.to_csv(out_csv, index=False)
                print(f"Loaded from Snowflake. Snapshot saved to {out_csv}")
                return df
        except Exception as e:
            print("Snowflake load failed, falling back to CSV:", e)

    # CSV fallback
    df = pd.read_csv(CSV_FALLBACK)
    df.columns = [c.lower() for c in df.columns]
    print(f"Loaded from CSV fallback: {CSV_FALLBACK}")
    return df

df = load_data()
print(df.shape)
df.head()


/tmp/ipykernel_20480/3971871104.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)


Loaded from Snowflake. Snapshot saved to ../docs/eda/int_annual_production_for_eda.csv
(224634, 16)


,api_well_number,reporting_year,months_in_production,oil_produced_bbl,gas_produced_mcf,water_produced_bbl,well_status_code,well_type_code,company_name,county,town,production_field,producing_formation,well_name,new_georeferenced_column,records_aggregated
0,31013191070000,2013,11,0.0,700.0,10.0,AC,GD,"Shelex Drilling, Inc.",Chautauqua,North Harmony,Lakeshore,Medina,Cheney Unit 2,"North Harmony, NY\n(42.094296, -79.374999)",1
1,31013126040000,2003,10,0.0,1504.0,0.0,AC,GD,Empire Energy E&P LLC,Chautauqua,Chautauqua,Lakeshore,Medina,Mulholland 509,"Chautauqua, NY\n(42.251704, -79.501693)",1
2,31009180710000,2006,12,0.0,448.0,0.0,AC,GD,"Texas Keystone, Inc.",Cattaraugus,Otto,Lakeshore,Medina,A Lumber 1,"Otto, NY\n(42.35589, -78.832619)",1
3,31013126840000,2016,12,0.0,1100.0,0.0,AC,GD,Waffenbauch USA Corp.,Chautauqua,Chautauqua,Lakeshore,Medina,Farver 1,"Chautauqua, NY\n(42.251704, -79.501693)",1
4,31009653900000,2006,12,0.0,0.0,252.0,AC,OD,"Little, Jay R.",Cattaraugus,Carrollton,Chipmunk,Chipmunk,Pfouts I 12,"Carrollton, NY\n(42.023289, -78.628438)",1


In [8]:

# Basic info
display(df.describe(include='all'))
missing = df.isna().mean().sort_values(ascending=False)
missing.to_csv(Path(EXPORT_DIR) / "missingness.csv", header=["null_ratio"])
missing.head(20)


,api_well_number,reporting_year,months_in_production,oil_produced_bbl,gas_produced_mcf,water_produced_bbl,well_status_code,well_type_code,company_name,county,town,production_field,producing_formation,well_name,new_georeferenced_column,records_aggregated
count,224634,224634.000000,224634.000000,224634.000000,2.246340e+05,224634.00000,224622,224634,224613,224622,224354,222416,220252,224614,224515,224634.000000
unique,17694,NaN,NaN,NaN,NaN,NaN,22,17,1626,32,305,278,72,16945,306,NaN
top,31013191070000,NaN,NaN,NaN,NaN,NaN,AC,GD,Empire Energy E&P LLC,Chautauqua,Allegany,Lakeshore,Medina,Fee 1,"Allegany, NY\n(42.088061, -78.491258)",NaN
freq,17,NaN,NaN,NaN,NaN,NaN,163860,116448,29655,71505,20090,70086,103648,621,20090,NaN
mean,NaN,2009.062030,8.360582,20.515214,2.623071e+03,58.69186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.001772
std,NaN,4.858323,5.096206,96.822100,4.794925e+04,715.33077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042055
min,NaN,2001.000000,0.000000,0.000000,0.000000e+00,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
25%,NaN,2005.000000,2.000000,0.000000,0.000000e+00,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
50%,NaN,2009.000000,12.000000,0.000000,1.290000e+02,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
75%,NaN,2013.000000,12.000000,0.000000,1.074000e+03,3.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


producing_formation         0.019507
production_field            0.009874
town                        0.001246
new_georeferenced_column    0.000530
company_name                0.000093
well_name                   0.000089
well_status_code            0.000053
county                      0.000053
well_type_code              0.000000
water_produced_bbl          0.000000
gas_produced_mcf            0.000000
oil_produced_bbl            0.000000
reporting_year              0.000000
months_in_production        0.000000
api_well_number             0.000000
records_aggregated          0.000000
dtype: float64

In [ ]:

numeric_cols = [
    c for c in ["months_in_production", "gas_produced_mcf", "water_produced_bbl", "oil_produced_bbl"]
    if c in df.columns
]

for col in numeric_cols:
    plt.figure(figsize=(8,5))
    df[col].dropna().hist(bins=50)
    plt.title(f"Histogram — {col}")
    plt.xlabel(col)
    plt.ylabel("count")
    out = Path(EXPORT_DIR) / f"hist_{col}.png"
    plt.tight_layout()
    plt.savefig(out, dpi=120) 
    plt.close()
    print("Saved", out)


Saved ../docs/eda/hist_months_in_production.png
Saved ../docs/eda/hist_gas_produced_mcf.png
Saved ../docs/eda/hist_water_produced_bbl.png
Saved ../docs/eda/hist_oil_produced_bbl.png


In [ ]:

year_col = "reporting_year"
agg_cols = [c for c in ["oil_produced_bbl", "gas_produced_mcf", "water_produced_bbl"] if c in df.columns]

totals_by_year = df.groupby(year_col)[agg_cols].sum().reset_index()
totals_by_year.to_csv(Path(EXPORT_DIR) / "totals_by_year.csv", index=False)
display(totals_by_year.head())

for col in agg_cols:
    plt.figure(figsize=(9,5))
    plt.plot(totals_by_year[year_col], totals_by_year[col], marker="o")
    plt.title(f"Total {col} by {year_col}")
    plt.xlabel("year")
    plt.ylabel(col)
    out = Path(EXPORT_DIR) / f"total_{col}.png"
    plt.tight_layout()
    plt.savefig(out, dpi=120)
    plt.close()
    print("Saved", out)


,reporting_year,oil_produced_bbl,gas_produced_mcf,water_produced_bbl
0,2001,86976.66,2.802064e+07,1319750.3
1,2002,86187.00,3.710751e+07,1426012.0
2,2003,161487.00,3.600388e+07,864990.0
3,2004,181230.00,4.694680e+07,1083793.0
4,2005,202301.00,5.516010e+07,1259321.0


Saved ../docs/eda/total_oil_produced_bbl.png
Saved ../docs/eda/total_gas_produced_mcf.png
Saved ../docs/eda/total_water_produced_bbl.png


In [ ]:

if "api_well_number" in df.columns and "oil_produced_bbl" in df.columns:
    top = (
        df.groupby("api_well_number", as_index=False)["oil_produced_bbl"]
          .sum()
          .sort_values("oil_produced_bbl", ascending=False)
          .head(20)
    )
    top.to_csv(Path(EXPORT_DIR) / "top20_wells_oil.csv", index=False)
    plt.figure(figsize=(9,6))
    plt.barh(top["api_well_number"].astype(str), top["oil_produced_bbl"])
    plt.gca().invert_yaxis()
    plt.title("Top-20 wells by total oil (bbl)")
    plt.xlabel("oil_produced_bbl")
    plt.ylabel("api_well_number")
    out = Path(EXPORT_DIR) / "top20_wells_oil.png"
    plt.tight_layout()
    plt.savefig(out, dpi=120) 
    plt.close()
    print("Saved", out)


Saved ../docs/eda/top20_wells_oil.png


In [ ]:

# Year-over-year oil decline per well (pct), clipped to [-1, 1] to stabilize tails
if set(["api_well_number","reporting_year","oil_produced_bbl"]).issubset(df.columns):
    srt = df.sort_values(["api_well_number","reporting_year"])
    srt["prev_oil"] = srt.groupby("api_well_number")["oil_produced_bbl"].shift(1)
    # Avoid div-by-zero and extreme spikes
    srt["decline_pct"] = (srt["oil_produced_bbl"] - srt["prev_oil"]) / srt["prev_oil"].replace({0: np.nan})
    clipped = srt["decline_pct"].clip(lower=-1, upper=1)
    plt.figure(figsize=(8,5))
    clipped.dropna().hist(bins=50)
    plt.title("Distribution of YoY oil decline (clipped to [-1,1])")
    plt.xlabel("decline_pct") 
    plt.ylabel("count")
    out = Path(EXPORT_DIR) / "hist_oil_decline_pct.png"
    plt.tight_layout()
    plt.savefig(out, dpi=120)
    plt.close()
    print("Saved", out)


Saved ../docs/eda/hist_oil_decline_pct.png


In [13]:

# Build an annual oil series for forecasting (aggregate, Prophet-friendly: ds/y)
if set(["reporting_year","oil_produced_bbl"]).issubset(df.columns):
    series = (
        df.groupby("reporting_year", as_index=False)["oil_produced_bbl"]
          .sum()
          .rename(columns={"reporting_year":"ds", "oil_produced_bbl":"y"})
    )
    series["ds"] = pd.to_datetime(series["ds"], format="%Y")
    out_csv = Path(EXPORT_DIR) / "oil_yearly_series.csv"
    series.to_csv(out_csv, index=False)
    display(series.tail())
    print("Saved", out_csv)


,ds,y
12,2013-01-01,313080.0
13,2014-01-01,355325.0
14,2015-01-01,279462.0
15,2016-01-01,220901.0
16,2017-01-01,192679.0


Saved ../docs/eda/oil_yearly_series.csv



## Next
- Continue with **H6.S2 (forecasting)** using `docs/eda/oil_yearly_series.csv`.
- Suggested models: baseline (naive), moving-average, and Prophet for annual series.
